In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f'Shape of inputs: {X.shape}')
    print(f'Shape of outputs: {y.shape}')
    break

Shape of inputs: torch.Size([64, 1, 28, 28])
Shape of outputs: torch.Size([64])


In [4]:
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)
print(f'Using {device} device')

Using cuda device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f'loss: {loss:>7f}\t[{current:>5d}/{size:>5d}]')

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \nAccuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n')


In [8]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
-------------------------------
loss: 2.306565	[   64/60000]
loss: 2.296334	[ 6464/60000]
loss: 2.288674	[12864/60000]
loss: 2.277187	[19264/60000]
loss: 2.249564	[25664/60000]
loss: 2.227471	[32064/60000]
loss: 2.228865	[38464/60000]
loss: 2.196858	[44864/60000]
loss: 2.197814	[51264/60000]
loss: 2.159690	[57664/60000]
Test Error: 
Accuracy: 41.9%, Avg loss: 2.159588

Epoch 2
-------------------------------
loss: 2.166950	[   64/60000]
loss: 2.159040	[ 6464/60000]
loss: 2.108345	[12864/60000]
loss: 2.118612	[19264/60000]
loss: 2.060430	[25664/60000]
loss: 2.006612	[32064/60000]
loss: 2.032229	[38464/60000]
loss: 1.958643	[44864/60000]
loss: 1.965605	[51264/60000]
loss: 1.878758	[57664/60000]
Test Error: 
Accuracy: 60.0%, Avg loss: 1.888336

Epoch 3
-------------------------------
loss: 1.919847	[   64/60000]
loss: 1.890754	[ 6464/60000]
loss: 1.781419	[12864/60000]
loss: 1.812826	[19264/60000]
loss: 1.697071	[25664/60000]
loss: 1.659887	[32064/60000]
loss: 1.677343	[38464/6000